In [1]:
!pip install -U langgraph langsmith langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.16
    Uninstalling langsmith-0.4.16:
      Successfully uninstalled langsmith-0.4.16
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchai

In [2]:
import getpass
import os

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from typing import TypedDict, List
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI


In [3]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")


Enter your Google AI API key: ··········


In [4]:
llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-thinking-exp-01-21",
        temperature=0.2
    )


In [5]:
from langgraph.channels import LastValue
from typing import TypedDict

# ✅ Correct usage
class ReviewState(TypedDict):
    proposal: LastValue[str]   # proposal is "last value" channel of type str
    legal_feedback: str
    ethics_feedback: str
    technical_feedback: str
    financial_feedback: str
    policy_feedback: str
    final_verdict: str


In [6]:
# Define agents with concise, structured prompts
def legal_advisor(state: ReviewState) -> ReviewState:
    print("Inside legal_advisor ")
    msg = f"""
You are a Legal Advisor.
Proposal: {state['proposal']}
Task: Identify legal risks under Indian Penal Code (IPC) and suggest modifications if needed.
Respond clearly with:
- Issues
- Suggested Fixes
- Verdict (Legal/Not Legal)
"""
    response = llm.invoke(msg).content
    state["legal_feedback"] = response
    return state


def ethics_expert(state: ReviewState) -> ReviewState:
    print("Inside ethics_expert ")

    msg = f"""
You are an Ethics Expert.
Proposal: {state['proposal']}
Task: Highlight ethical concerns and fairness issues.
Respond with:
- Key Ethical Risks
- Mitigation Suggestions
- Verdict (Ethical/Not Ethical)
"""
    response = llm.invoke(msg).content
    state["ethics_feedback"] = response
    return state


def financial_analyst(state: ReviewState) -> ReviewState:
    print("Inside financial_analyst ")

    msg = f"""
You are a Financial Analyst.
Proposal: {state['proposal']}
Task: Assess costs, ROI, and sustainability.
Respond with:
- Estimated Costs
- ROI & Risks
- Verdict (Financially Viable/Not Viable)
"""
    response = llm.invoke(msg).content
    state["financial_feedback"] = response
    return state


def technical_analyst(state: ReviewState) -> ReviewState:
    print("Inside technical_analyst ")

    msg = f"""
You are a Technical Analyst.
Proposal: {state['proposal']}
Task: Evaluate feasibility, technical risks, and limitations.
Respond with:
- Feasibility Issues
- Known Limitations
- Verdict (Feasible/Not Feasible)
"""
    response = llm.invoke(msg).content
    state["technical_feedback"] = response
    return state


def final_verdict(state: ReviewState) -> ReviewState:
    print("Inside final_verdict ")

    summary_prompt = f"""
Board Review Summary:

Legal: {state['legal_feedback']}
Ethics: {state['ethics_feedback']}
Financial: {state['financial_feedback']}
Technical: {state['technical_feedback']}

Task: Provide a final verdict.
Answer with:
- Overall Risk Level (Low/Medium/High)
- Deployment Recommendation (Yes/No)
- Short Rationale
"""
    response = llm.invoke(summary_prompt).content
    state["final_verdict"] = response
    return state


In [7]:
# Build LangGraph
builder = StateGraph(ReviewState)

builder.add_node("legal_advisor", legal_advisor)
builder.add_node("ethics_expert", ethics_expert)
builder.add_node("financial_analyst", financial_analyst)
builder.add_node("technical_analyst", technical_analyst)
builder.add_node("final_verdict", final_verdict)
builder.add_node("technical_analyst2", technical_analyst)




builder.set_entry_point("technical_analyst")

# Parallel paths

builder.add_edge("technical_analyst", "ethics_expert")
builder.add_edge("ethics_expert", "legal_advisor")
builder.add_edge("legal_advisor", "financial_analyst")
builder.add_edge("financial_analyst", "technical_analyst2")
builder.add_edge("technical_analyst2", "final_verdict")
builder.add_edge("final_verdict", END)

# Compile
graph = builder.compile()


In [8]:
# Run!
if __name__ == "__main__":
    user_input ="""
    We want to deploy an AI system in public schools that uses students' webcam footage during online exams to
    detect facial expressions and predict cheating behavior. The system flags suspicious behavior for review
    by school authorities.
    """

    initial_state = {
        "proposal": user_input,
        "legal_feedback": "",
        "ethics_feedback": "",
        "financial_feedback": "",
        "technical_feedback": "",
        "final_verdict": ""
    }
    result = graph.invoke(initial_state)

    print("\n🧾 Final Verdict:\n", result["final_verdict"])


Inside technical_analyst 
Inside ethics_expert 
Inside legal_advisor 
Inside financial_analyst 
Inside technical_analyst 
Inside final_verdict 

🧾 Final Verdict:
 **Overall Risk Level: High**

**Deployment Recommendation: No**

**Short Rationale:** The proposed AI system is fundamentally flawed and poses unacceptable risks across all evaluated dimensions. It is likely illegal under Indian law (infringing on the right to privacy and potentially voyeurism), profoundly unethical due to pervasive surveillance and psychological harm to minors, financially unviable given its high costs and lack of tangible ROI, and technically unfeasible due to inherent inaccuracies, biases, and infrastructure demands. The severe legal, ethical, and reputational liabilities far outweigh any perceived benefits.


In [10]:
# Run!
if __name__ == "__main__":
    user_input ="""
We propose a city-wide smart traffic light system that adjusts signals using AI predictions of traffic flow. It requires installation of cameras at major intersections.
    """

    initial_state = {
        "proposal": user_input,
        "legal_feedback": "",
        "ethics_feedback": "",
        "financial_feedback": "",
        "technical_feedback": "",
        "final_verdict": ""
    }
    result = graph.invoke(initial_state)

    print("\n🧾 Final Verdict:\n", result["final_verdict"])


Inside technical_analyst 
Inside ethics_expert 
Inside legal_advisor 
Inside financial_analyst 
Inside technical_analyst 
Inside final_verdict 

🧾 Final Verdict:
 **Final Verdict**

*   **Overall Risk Level:** High
*   **Deployment Recommendation:** No (for immediate, full city-wide deployment)
*   **Short Rationale:** While technically feasible and offering potential benefits, the proposal for immediate city-wide deployment carries substantial legal, ethical, financial, and technical risks. These include significant privacy and surveillance concerns, potential algorithmic bias, immense upfront and ongoing costs, complex integration challenges, and critical cybersecurity vulnerabilities. A phased approach, starting with a detailed feasibility study and a small-scale pilot project with robust safeguards and clear accountability, is strongly recommended before considering broader implementation.


In [12]:
# Run!
if __name__ == "__main__":
    user_input ="""
We propose an AI system in hospitals that analyzes X-ray and MRI scans to assist doctors in detecting early signs of tuberculosis. The system will only be used as a support tool, not as a replacement for doctors.
    """

    initial_state = {
        "proposal": user_input,
        "legal_feedback": "",
        "ethics_feedback": "",
        "financial_feedback": "",
        "technical_feedback": "",
        "final_verdict": ""
    }
    result = graph.invoke(initial_state)

    print("\n🧾 Final Verdict:\n", result["final_verdict"])


Inside technical_analyst 
Inside ethics_expert 
Inside legal_advisor 
Inside financial_analyst 
Inside technical_analyst 
Inside final_verdict 

🧾 Final Verdict:
 **Overall Risk Level:** High

**Deployment Recommendation:** Yes, with Extreme Caution and Strict Adherence to Safeguards

**Short Rationale:**
The proposed AI system holds significant potential for improving public health by enabling earlier detection of tuberculosis. Technically, it is feasible, and ethically, its intent as a support tool is sound. However, the path to deployment is fraught with **high legal, ethical, and financial risks**. Criminal negligence, patient safety (false negatives), algorithmic bias, data privacy, and substantial development/regulatory costs are major concerns.

Deployment is recommended **only if and when** all suggested safeguards – including rigorous clinical validation, transparent disclosure of limitations, robust SOPs mandating independent doctor review, comprehensive training, continuous 

In [13]:
print("\n🧾 Legal Feedback:\n", result["legal_feedback"])
print("\n🧾 Ethics Feedback:\n", result["ethics_feedback"])
print("\n🧾 Financial Feedback:\n", result["financial_feedback"])
print("\n🧾 Technical Feedback:\n", result["technical_feedback"])


🧾 Legal Feedback:
 As your Legal Advisor, I have reviewed your proposal for an AI system in hospitals to assist doctors in detecting early signs of tuberculosis from X-ray and MRI scans. While the intent is commendable and the potential benefits significant, it is crucial to identify and mitigate legal risks, particularly under the Indian Penal Code (IPC).

The core principle here is that the AI is a "support tool, not a replacement for doctors." This distinction is vital in determining liability.

---

### Issues under Indian Penal Code (IPC)

The primary risks under IPC revolve around criminal negligence, where an act or omission causes harm or death due to a lack of reasonable care.

1.  **Criminal Negligence Leading to Death (Section 304A IPC):**
    *   **Scenario:** If the AI system provides an incorrect analysis (e.g., a "false negative" missing early signs of TB), and a doctor, relying on this output, fails to diagnose or delays treatment, leading to the patient's death.
    *